## 30s k-fold

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, AveragePooling1D, Dropout
from keras.layers import Activation, BatchNormalization
from keras.optimizers import Adam
from keras.utils import np_utils
import tensorflow as tf
from keras.callbacks import TensorBoard, EarlyStopping
np.set_printoptions(suppress=True)

trainD = np.load("/home/hsiehch/30s/train_data.npy")
trainL = np.load("/home/hsiehch/30s/train_label.npy")
validationD = np.load("/home/hsiehch/30s/validation_data.npy")
validationL = np.load("/home/hsiehch/30s/validation_label.npy")
testD = np.load("/home/hsiehch/30s/test_data.npy")
testL = np.load("/home/hsiehch/30s/test_label.npy")

trainD = np.append(trainD, validationD, axis=0)
trainL = np.append(trainL, validationL, axis=0)
trainD = np.append(trainD, testD, axis=0)
trainL = np.append(trainL, testL, axis=0)

trainData = trainD.reshape((trainD.shape[0], trainD.shape[1], 1))
trainLabel = np_utils.to_categorical(trainL, 4)
print('Train Data:', trainData.shape)
print('Train Label: ', trainLabel.shape)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train Data: (10151, 9000, 1)
Train Label:  (10151, 4)


In [2]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True)
print(kf)

KFold(n_splits=5, random_state=None, shuffle=True)


In [3]:
from sklearn.metrics import f1_score
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

def create_model(learning_rate=0):
    model = Sequential() 
    model.add(Conv1D(filters = 32, kernel_size = 7, input_shape = (trainData.shape[1], 1)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 32, kernel_size = 7))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 64, kernel_size = 7))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 64, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 128, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 128, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 256, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 256, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 512, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 512, kernel_size = 3))
    model.add(Activation('relu'))

    model.add(Flatten())
    model.add(Dense(128, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation = 'relu'))
    model.add(Dense(4, activation = "softmax"))
    
    adam = Adam(lr = learning_rate)
    model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics=['accuracy'])
    
    return model


batch_S = [30, 50, 70, 90, 120]
learning_rate = [0.00005, 0.0001, 0.0005, 0.001, 0.005]
parameter_grid = dict(batch_size=batch_S, learning_rate=learning_rate)
model = KerasClassifier(build_fn = create_model, epochs=80, verbose=1)
grid = GridSearchCV(estimator=model, param_grid=parameter_grid, cv=kf, verbose = 1)
grid_result = grid.fit(trainData, trainLabel)

print('Finish training!')

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Epoch 1/80
8120/8120 [==============================] - 16s 2ms/step - loss: 1.0604 - acc: 0.5765
Epoch 2/80
8120/8120 [==============================] - 8s 1ms/step - loss: 0.9939 - acc: 0.5857
Epoch 3/80
8120/8120 [==============================] - 9s 1ms/step - loss: 0.9409 - acc: 0.5899
Epoch 4/80
8120/8120 [==============================] - 9s 1ms/step - loss: 0.9015 - acc: 0.6107
Epoch 5/80
8120/8120 [==============================] - 9s 1ms/step - loss: 0.8753 - acc: 0.6293
Epoch 6/80
8120/8120 [==============================] - 9s 1ms/step - loss: 0.8481 - acc: 0.6422
Epoch 7/80
8120/8120 [==============================] - 9s 1ms/step - loss: 0.8211 - acc: 0.6521
Epoch 8/80
8120/8120 [==============================] - 9s 1ms/step - loss: 0.7990 - acc: 0.6606
Epoch 9/80
8120/8120 [==============================] - 9s 1ms/step - loss: 0.7723 - acc: 0.6760
Epoch 10/80
8120/8120 [==============================] - 9s 1ms/

8121/8121 [==============================] - 8s 1ms/step - loss: 0.9326 - acc: 0.5907
Epoch 5/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.8883 - acc: 0.6158
Epoch 6/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.8517 - acc: 0.6318
Epoch 7/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.8297 - acc: 0.6387
Epoch 8/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.8108 - acc: 0.6515
Epoch 9/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.7883 - acc: 0.6627
Epoch 10/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.7741 - acc: 0.6686
Epoch 11/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.7552 - acc: 0.6773
Epoch 12/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.7313 - acc: 0.6871
Epoch 13/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.7065 - acc: 0.6975
Epoch 14/80
8121/8121 [==============

8121/8121 [==============================] - 9s 1ms/step - loss: 0.8234 - acc: 0.6560
Epoch 9/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.7920 - acc: 0.6691
Epoch 10/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.7751 - acc: 0.6744
Epoch 11/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.7564 - acc: 0.6798
Epoch 12/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.7381 - acc: 0.6866
Epoch 13/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.7056 - acc: 0.7008
Epoch 14/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.6964 - acc: 0.7039
Epoch 15/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.6766 - acc: 0.7136
Epoch 16/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.6610 - acc: 0.7173
Epoch 17/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.6414 - acc: 0.7243
Epoch 18/80
8121/8121 [==========

8121/8121 [==============================] - 9s 1ms/step - loss: 0.5463 - acc: 0.7877
Epoch 13/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.5354 - acc: 0.7952
Epoch 14/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.5219 - acc: 0.8050
Epoch 15/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.5135 - acc: 0.8042
Epoch 16/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.5008 - acc: 0.8084
Epoch 17/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.4833 - acc: 0.8203
Epoch 18/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.4814 - acc: 0.8191
Epoch 19/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.4616 - acc: 0.8259
Epoch 20/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.4614 - acc: 0.8240
Epoch 21/80
8121/8121 [==============================] - 8s 1ms/step - loss: 0.4624 - acc: 0.8239
Epoch 22/80
8121/8121 [=========

8121/8121 [==============================] - 9s 1ms/step - loss: 0.5235 - acc: 0.7955
Epoch 17/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.5061 - acc: 0.8068
Epoch 18/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.4952 - acc: 0.8083
Epoch 19/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.4898 - acc: 0.8168
Epoch 20/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.4824 - acc: 0.8203
Epoch 21/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.4676 - acc: 0.8208
Epoch 22/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.4631 - acc: 0.8291
Epoch 23/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.4525 - acc: 0.8327
Epoch 24/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.4464 - acc: 0.8295
Epoch 25/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.4391 - acc: 0.8376
Epoch 26/80
8121/8121 [=========

...

Epoch 16/80
8120/8120 [==============================] - 8s 952us/step - loss: 0.9933 - acc: 0.5871
Epoch 17/80
8120/8120 [==============================] - 8s 951us/step - loss: 0.9927 - acc: 0.5871
Epoch 18/80
8120/8120 [==============================] - 8s 946us/step - loss: 0.9929 - acc: 0.5871
Epoch 19/80
8120/8120 [==============================] - 8s 951us/step - loss: 0.9933 - acc: 0.5871
Epoch 20/80
8120/8120 [==============================] - 8s 961us/step - loss: 0.9933 - acc: 0.5871
Epoch 21/80
8120/8120 [==============================] - 8s 953us/step - loss: 0.9929 - acc: 0.5871
Epoch 22/80
8120/8120 [==============================] - 8s 947us/step - loss: 0.9931 - acc: 0.5871
Epoch 23/80
8120/8120 [==============================] - 8s 945us/step - loss: 0.9931 - acc: 0.5871
Epoch 24/80
8120/8120 [==============================] - 8s 956us/step - loss: 0.9924 - acc: 0.5871
Epoch 25/80
8120/8120 [==============================] - 8s 960us/step - loss: 0.9925 - acc: 0.5871


Epoch 17/80
8121/8121 [==============================] - 8s 943us/step - loss: 0.9971 - acc: 0.5865
Epoch 18/80
8121/8121 [==============================] - 8s 943us/step - loss: 0.9972 - acc: 0.5865
Epoch 19/80
8121/8121 [==============================] - 8s 951us/step - loss: 0.9972 - acc: 0.5865
Epoch 20/80
8121/8121 [==============================] - 8s 962us/step - loss: 0.9972 - acc: 0.5865
Epoch 21/80
8121/8121 [==============================] - 8s 959us/step - loss: 0.9969 - acc: 0.5865
Epoch 22/80
8121/8121 [==============================] - 8s 958us/step - loss: 0.9975 - acc: 0.5865
Epoch 23/80
8121/8121 [==============================] - 8s 939us/step - loss: 0.9972 - acc: 0.5865
Epoch 24/80
8121/8121 [==============================] - 8s 953us/step - loss: 0.9972 - acc: 0.5865
Epoch 25/80
8121/8121 [==============================] - 8s 938us/step - loss: 0.9975 - acc: 0.5865
Epoch 26/80
8121/8121 [==============================] - 8s 949us/step - loss: 0.9976 - acc: 0.5865


Epoch 18/80
8121/8121 [==============================] - 8s 938us/step - loss: 0.9806 - acc: 0.5869
Epoch 19/80
8121/8121 [==============================] - 8s 946us/step - loss: 0.9892 - acc: 0.5869
Epoch 20/80
8121/8121 [==============================] - 8s 941us/step - loss: 0.9893 - acc: 0.5869
Epoch 21/80
8121/8121 [==============================] - 8s 951us/step - loss: 0.9886 - acc: 0.5869
Epoch 22/80
8121/8121 [==============================] - 8s 948us/step - loss: 0.9893 - acc: 0.5869
Epoch 23/80
8121/8121 [==============================] - 8s 934us/step - loss: 0.9891 - acc: 0.5869
Epoch 24/80
8121/8121 [==============================] - 8s 956us/step - loss: 0.9890 - acc: 0.5869
Epoch 25/80
8121/8121 [==============================] - 8s 954us/step - loss: 0.9888 - acc: 0.5869
Epoch 26/80
8121/8121 [==============================] - 8s 958us/step - loss: 0.9888 - acc: 0.5869
Epoch 27/80
8121/8121 [==============================] - 8s 944us/step - loss: 0.9901 - acc: 0.5869


[Parallel(n_jobs=1)]: Done 125 out of 125 | elapsed: 1325.1min finished


Epoch 1/80
10151/10151 [==============================] - 28s 3ms/step - loss: 0.9945 - acc: 0.5725
Epoch 2/80
10151/10151 [==============================] - 11s 1ms/step - loss: 0.8411 - acc: 0.6161
Epoch 3/80
10151/10151 [==============================] - 11s 1ms/step - loss: 0.7482 - acc: 0.6844
Epoch 4/80
10151/10151 [==============================] - 11s 1ms/step - loss: 0.6915 - acc: 0.7212
Epoch 5/80
10151/10151 [==============================] - 11s 1ms/step - loss: 0.6481 - acc: 0.7433
Epoch 6/80
10151/10151 [==============================] - 11s 1ms/step - loss: 0.6014 - acc: 0.7646
Epoch 7/80
10151/10151 [==============================] - 11s 1ms/step - loss: 0.5691 - acc: 0.7798
Epoch 8/80
10151/10151 [==============================] - 11s 1ms/step - loss: 0.5345 - acc: 0.7979
Epoch 9/80
10151/10151 [==============================] - 11s 1ms/step - loss: 0.5395 - acc: 0.7956
Epoch 10/80
10151/10151 [==============================] - 11s 1ms/step - loss: 0.5374 - acc: 0.7960

In [4]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean:%f (STD:%f) with: %r" % (mean, stdev, param))

Mean:0.835090 (STD:0.007316) with: {'learning_rate': 5e-05, 'batch_size': 30}
Mean:0.840902 (STD:0.005353) with: {'learning_rate': 0.0001, 'batch_size': 30}
Mean:0.845040 (STD:0.006316) with: {'learning_rate': 0.0005, 'batch_size': 30}
Mean:0.842577 (STD:0.009344) with: {'learning_rate': 0.001, 'batch_size': 30}
Mean:0.587036 (STD:0.003958) with: {'learning_rate': 0.005, 'batch_size': 30}
Mean:0.830854 (STD:0.009316) with: {'learning_rate': 5e-05, 'batch_size': 50}
Mean:0.839819 (STD:0.007986) with: {'learning_rate': 0.0001, 'batch_size': 50}
Mean:0.844547 (STD:0.003870) with: {'learning_rate': 0.0005, 'batch_size': 50}
Mean:0.845237 (STD:0.006778) with: {'learning_rate': 0.001, 'batch_size': 50}
Mean:0.587036 (STD:0.003958) with: {'learning_rate': 0.005, 'batch_size': 50}
Mean:0.818146 (STD:0.008648) with: {'learning_rate': 5e-05, 'batch_size': 70}
Mean:0.840114 (STD:0.008227) with: {'learning_rate': 0.0001, 'batch_size': 70}
Mean:0.843759 (STD:0.006813) with: {'learning_rate': 0.0005

In [5]:
grid_result.best_score_

0.8457294779450492

In [6]:
grid_result.best_params_

{'batch_size': 70, 'learning_rate': 0.001}